In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
tp = pd.read_csv(r'C:\Users\avelurta\Documents\UTN\Ciencia de Datos\TP\Ecommerce ABI Enriched.csv', delimiter=',', low_memory= 'false')

C:\Users\avelurta\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (21,29,47,62,68,74,82,84,91,103,104,112,126) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
tp.head(30)
tp.shape

(359388, 141)

In [26]:
cant_NaN = tp.isnull().sum()
print(cant_NaN,'\r\n')

age                            359388
birth_date                     359388
birth_date_day                 359388
birth_date_month               359388
birth_date_year                359388
channel                        359388
city                           359388
complement                     359388
country                             0
created_in                          0
creation_brand                      0
district                       359388
document_id                    359388
document_id_unify              359388
email                          359388
emails                         359388
facebook_id                    359388
first_name                     359388
full_name                      359388
gender                         359388
id                                  0
id_1                           133049
identification_number          359388
identification_number_unify    359388
is_employee                    359388
last_name                      359388
mobile_numbe

In [27]:
tp['brand'].unique()

array(['Goose Island', 'Lowenbrau', 'unknown', "Beck's", 'Budweiser',
       'Del Puerto', 'Stella Artois', 'Leffe', 'Corona',
       'Cerveza Quimera', 'Hertog Jan', 'Quimera', 'Cusqueña',
       'La Chouffe', 'Vedett White', nan, 'Casa de la Cerveza', 'Duvel',
       'Franziskaner', 'Hoegaarden', 'Spaten', 'Golden Road',
       'Ten Barrel', 'Belle Vue', 'Elysian', 'Liefmans', 'Four Peaks',
       'Mongozo', 'Modelo', 'Grupo Modelo', 'Patagonia Brewing Co.',
       'Flensburger Brauerei', 'Peñón del Águila', 'BevyBar',
       'Multimarca', 'Spaten-Franziskaner-Bräu', 'BBC',
       'Anchor Brewing Company', 'Bass Brewery', 'Hofbräu München',
       'Adnams', "Fuller's", 'Karlsberg Brauerei',
       'Les Brasseurs de Gayant\xa0', 'Craft Society',
       'Brouwerij Van Steenberge', 'Fuegian Beverage Company',
       'De La Vaca Tranquila Cervezas Artesanales', 'Tsingtao',
       'Löwenbräu', 'Barba Roja', 'Cervecería y Maltería Quilmes',
       'Coopers Brewery', 'Backus y Johnston', 'B

In [128]:
tp_f = a.drop(['creation_brand','opt_in',
       'origin_score', 'system', 'table_origin', 'td_id', 'title', 'id_4',
       'time', 'td_id.1', 'customer_first_order_period', 'product_spk', 'business_spk', 'unit_discount_usd',
       'source_system', 'business_name','unit_tax_usd','unit_delivery_charged_usd',
       'currency', 'customer_first_order_year', 'unit_net_revenue_usd','order_status_type', 'customer_spk','unit_gross_revenue_usd',
       'source_system_order_id','unit_price_charged_usd','unit_tax_local', 'source','department','weight', 'product_nk','is_abi',
       'source_schema', 'time.1'], axis = 1)

In [129]:
tp_f = tp_f.loc[tp_f['country'] == 'Argentina']

In [130]:
tp_f.head(50)

,country,created_in,id,id_1,unit_discount_local,coupon_code,customer_first_order_date,elegible_discount_spk,unit_gross_revenue_local,delivery_address_spk,...,row_updated_at,item_id,sku,brand,date_created,image_url,upc,name,category,subcategory
17160,Argentina,"[""2018-01-12""]","[""ar_bevybar;1b427493e7cc5228ccfdb5f70253e7ef1...",NaN,0.0000,NaN,2018-01-12,NaN,1810.0000,NaN,...,NaN,3064163,CRW-BP-14,Grupo Modelo,2016-04-18 10:42:34,NaN,NaN,Pack Conservadora Corona,Mixed Pack,unknown
17161,Argentina,"[""2016-05-16""]","[""ar_bevybar;f14a46180a983653d1e151da458045331...",NaN,0.0000,NaN,2016-05-16,NaN,813.0000,NaN,...,NaN,3036599,6109617020,Patagonia Brewing Co.,2016-05-10 14:23:04,NaN,NaN,Pack Colección Patagonia XL,Mixed Pack,unknown
17162,Argentina,"[""2017-09-20""]","[""ar_bevybar;8b3eb4b88a85ba3eeb60dd24686995c38...",NaN,0.0000,NaN,2017-09-20,NaN,1030.0000,NaN,...,NaN,3036032,6105600014,Corona,2017-05-29 09:29:41,NaN,NaN,Pack Corona Experience,Beer,Others
17163,Argentina,"[""2018-04-18""]","[""ar_bevybar;9963d5cb526be2da8348e97b810925743...",NaN,0.0000,NaN,2018-04-18,NaN,1914.0000,NaN,...,NaN,3064163,CRW-BP-14,Grupo Modelo,2016-04-18 10:42:34,NaN,NaN,Pack Conservadora Corona,Mixed Pack,unknown
17164,Argentina,"[""2016-12-12""]","[""ar_bevybar;83c601952cc40eb2355a4ea33c7bb4bb9...",NaN,100.0000,degustar,2016-12-12,NaN,1035.0000,NaN,...,NaN,3036493,4003101006,Stella Artois,2016-02-24 19:07:55,NaN,QUI9000000008,Copas Stella Artois 250ml - Pack 6 unidades,Mixed Pack,unknown
17165,Argentina,"[""2017-06-13""]","[""ar_bevybar;41434b744330c691cf83b70da3fa080c8...",NaN,0.0000,NaN,2017-06-13,NaN,631.0000,NaN,...,NaN,3035776,6109610010,Patagonia Brewing Co.,2017-01-04 11:47:02,NaN,None,Pack de Cervezas y Vasos Patagonia,Mixed Pack,unknown
17166,Argentina,"[""2018-01-26""]","[""ar_bevybar;4680bed8642624a4cc66ae5b12c259da9...",NaN,0.0000,NaN,2018-01-26,NaN,112.5066,NaN,...,NaN,3036897,1431404024,Flensburger Brauerei,2017-12-27 10:28:26,https://cdn.shopify.com/s/files/1/1103/5152/pr...,NaN,Cerveza Flensburger Weizen,Beer,unknown
17167,Argentina,"[""2018-01-26""]","[""ar_bevybar;4680bed8642624a4cc66ae5b12c259da9...",NaN,0.0000,NaN,2018-01-26,NaN,724.4934,NaN,...,NaN,3036368,1300612024,Patagonia Brewing Co.,2017-10-30 12:03:15,NaN,7792798000418,Cerveza Patagonia 24.7 - Session IPA con Sauco...,Beer,unknown
17168,Argentina,"[""2018-01-26""]","[""ar_bevybar;4680bed8642624a4cc66ae5b12c259da9...",NaN,0.0000,NaN,2018-01-26,NaN,1452.0000,NaN,...,NaN,3036368,1300612024,Patagonia Brewing Co.,2017-10-30 12:03:15,NaN,7792798000418,Cerveza Patagonia 24.7 - Session IPA con Sauco...,Beer,unknown
17169,Argentina,"[""2018-05-19""]","[""ar_bevybar;9d3d65f4eccf1cd5d930f0dfb31ded597...",NaN,0.0000,NaN,2018-05-19,NaN,1771.0000,NaN,...,NaN,3064168,6106600013,Corona,2018-04-23 17:30:52,NaN,NaN,Pack Conservadora Corona,Mixed Pack,unknown


In [131]:
tp_f.shape

(133049, 35)

In [132]:
tp_f.columns

Index(['country', 'created_in', 'id', 'id_1', 'unit_discount_local',
       'coupon_code', 'customer_first_order_date', 'elegible_discount_spk',
       'unit_gross_revenue_local', 'delivery_address_spk', 'order_date',
       'order_status', 'discount_name', 'unit_price_charged_local',
       'unit_delivery_charged_local', 'rate_local_do_usd', 'operator_nk',
       'quantity', 'delivery_type', 'unit_net_revenue_local', 'discount_spk',
       'order_days_after_first_order', 'order_time', 'row_created_at',
       'local_category', 'row_updated_at', 'item_id', 'sku', 'brand',
       'date_created', 'image_url', 'upc', 'name', 'category', 'subcategory'],
      dtype='object')

In [133]:
tp_f.brand.value_counts()

Stella Artois                                26870
Patagonia Brewing Co.                        19573
BevyBar                                      17801
Corona                                        9376
Grupo Modelo                                  8025
Leffe                                         7313
Spaten-Franziskaner-Bräu                      6707
Hoegaarden                                    2972
Brauerei Beck & Co                            2290
Cervecería y Maltería Quilmes                 1688
Multimarca                                    1687
Löwenbräu                                     1311
De La Vaca Tranquila Cervezas Artesanales     1276
Craft Society                                 1261
Brouwerij Van Steenberge                      1183
Bass Brewery                                  1118
Peñón del Águila                               975
Coopers Brewery                                916
Les Brasseurs de Gayant                        902
Paulaner Brauerei München      

In [134]:
tp_f.shape

(133049, 35)

In [ ]:
## Algunos comentarios: 
##   No hay info de 2019, termina en 18 Agosto 2018
##   id podriamos usarlo para saber cuanto gasta una persona en promedio en ecommerce, ver evolucion
##   Bevybar y Craft Society son cosas sin marca, habria que unirlas


In [ ]:
##Analizo compras de usuarios 
plt.figure(figsize=(15,3))
g = sns.countplot(x="id",data=tp_f, palette = "muted")
g.set_xticklabels(g.get_xticklabels(), rotation=90, ha="right")
plt.title('Cantidad de viajes por usuario',size = 20)
plt.xlabel("Usuario")
plt.ylabel("Count")
plt.show()